<a href="https://colab.research.google.com/github/Linux-Server/DeepLearning.AI/blob/hugging_face/01_AutoTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import pipeline

In [52]:
classifier = pipeline("sentiment-analysis")
raw_inputs = ["I love you so much buddy", "I hate you"]
classifier(raw_inputs)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998651742935181},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

Above action of pipeline function is divided into three : Tokenizer(Preproceesing), Model, PostProcessing

First lets see how Tokenizer Works

In [14]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [32]:
# Now created the raw_inputs
raw_inputs = ["I love you so much buddy", "I hate you"]

Since we have created the AutoTokenizer from the checkpoint, we need to feed the tokenizer with thw raw input, which will return a dict, with inputs ids and attension mask

In [33]:
# without padding and truc
input_test = tokenizer(raw_inputs)
print("Input Test: ", input_test)
# We can also pass trucation and padding along with retun tensor type
inputs = tokenizer(raw_inputs, truncation=True, padding=True, return_tensors ="pt")
print("Input Test: ", inputs)


Input Test:  {'input_ids': [[101, 1045, 2293, 2017, 2061, 2172, 8937, 102], [101, 1045, 5223, 2017, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}
Input Test:  {'input_ids': tensor([[ 101, 1045, 2293, 2017, 2061, 2172, 8937,  102],
        [ 101, 1045, 5223, 2017,  102,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0]])}


In [34]:
# Now lets pass the output of tonezier to model
# We can do the same process
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [42]:
outputs = model(**inputs)
outputs.last_hidden_state.shape

torch.Size([2, 8, 768])

In [49]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
output = model(**inputs)
output.logits

tensor([[-4.2712,  4.6400],
        [ 3.8724, -3.1543]], grad_fn=<AddmmBackward0>)

In [50]:
# Next is Postprocessing the logits
# the logits are unmormalized raw outputs
# Inorder to convert this into huyman readable, it should go inside the softmax layer
model.config.id2label


{0: 'NEGATIVE', 1: 'POSITIVE'}

In [53]:
# To load softmax layer
import torch

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
predictions

tensor([[1.3485e-04, 9.9987e-01],
        [9.9911e-01, 8.8707e-04]], grad_fn=<SoftmaxBackward0>)

In [55]:
print(type(model))

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>
